# Eniviroment setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import vit_b_16
from torchvision.transforms import transforms

from dataset.dataset_info import ClassifierDatasetInfo, DatasetInfo
from dataset.dataset_type import DatasetType
from dataset.STARCOP_dataset import STARCOPDataset

from models.Tools.FilesHandler.model_files_handler import ModelFilesHandler
from models.Tools.Measures.measure_tool_factory import MeasureToolFactory
from models.Tools.Measures.model_type import ModelType

import sys
import os
from typing import Type

os.chdir(r"C:\Users\mpilc\Desktop\Studia\Thesis\Repozytoria\Thesis")

## Setup datasets
STARCOPDataset is custom class that derives torch.utils.data.Dataset class. It's defined in dataset module.

In [2]:
def setup_dataloaders(
        data_path: str = r"data",
        batch_size: int = 32,
        train_type = DatasetType.EASY_TRAIN,
        image_info_class: Type[DatasetInfo] = ClassifierDatasetInfo,
        crop_size: int = 1
):
    train_dataset = STARCOPDataset(
        data_path=data_path,
        data_type=train_type,
        image_info_class=image_info_class,
        crop_size=crop_size
    )
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = STARCOPDataset(
        data_path=data_path,
        data_type=DatasetType.TEST,
        image_info_class=image_info_class,
        crop_size=crop_size
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

## Setup models

### Model class

In [3]:
class CustomViT(nn.Module):
    def __init__(self, num_channels=9, num_classes=2):
        super(CustomViT, self).__init__()
        # Load pre-trained ViT model
        self.vit = vit_b_16(weights=None)  # Use pretrained weights if desired

        # Modify the input embedding layer to accept `num_channels`
        self.vit.conv_proj = nn.Conv2d(num_channels, self.vit.conv_proj.out_channels,
                                       kernel_size=self.vit.conv_proj.kernel_size,
                                       stride=self.vit.conv_proj.stride,
                                       padding=self.vit.conv_proj.padding,
                                       bias=(self.vit.conv_proj.bias is not None))

        # Modify the classifier head for binary classification
        self.vit.heads = nn.Sequential(
            nn.Linear(self.vit.heads.head.in_features, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        
        self.resize = nn.Upsample(size=(224, 224), mode='bilinear', align_corners=False)
        self.normalize = transforms.Normalize(mean=[0.5] * num_channels, std=[0.5] * num_channels)        

    def forward(self, x):
        print(x.shape)
        x = self.resize(x)
        x = self.normalize(x)
        print(x.shape)
        return self.vit(x)

### Prepare models 

In [4]:
def setup_model(model: nn.Module, lr: float, device: str):
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()  # Binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer

## Prepare training function

In [5]:
def print_progress_bar(percentage, loss):
    bar_length = 50  # Length of the progress bar
    filled_length = int(bar_length * percentage // 100)
    bar = '=' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f"\r[{bar}] {percentage:.2f}% [Loss: {loss:.6f}]")
    sys.stdout.flush()

In [6]:
def train(criterion, device, epochs, model, optimizer, dataloader, model_handler, log_batches: bool = False):
    model.train()
    for epoch in range(epochs):  
        print(f"Epoch: {epoch}")
        running_loss = 0.0
        for batch_id, (images, mag1c, labels) in enumerate(dataloader):
            optimizer.zero_grad()

            input_image = torch.cat((images, mag1c), dim=1).to(device)
            labels = labels.long().to(device)

            outputs = model(input_image)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if log_batches and (batch_id + 1) % 10 == 0:
                print_progress_bar(batch_id / len(dataloader) * 100, running_loss / (batch_id + 1))

        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")
        model_handler.save_raw_model(model)

## Prepare evaluate function

In [7]:
def evaluate(criterion, device, model, dataloader, measurer):
    model.eval()
    all_predictions = []
    all_labels = []
    running_loss = 0.0

    for batch_id, (images, mag1c, labels) in enumerate(dataloader):
        input_image = torch.cat((images, mag1c), dim=1).to(device)
        labels = labels.long().to(device)

        outputs = model(input_image)
        predictions = torch.argmax(outputs, dim=1)
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        all_predictions.append(predictions.cpu().detach())
        all_labels.append(labels.cpu().detach())

    measures = measurer.compute_measures(torch.cat(all_predictions), torch.cat(all_labels))
    print(f"Validation loss: {running_loss / len(dataloader)}.\nMeasures:\n{measures}")
    return measures

# Train model 

In [8]:
epochs = 15
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-4

# Override for file handler
from models.Transformer.VIT.model import CustomViT

train_dataloader, test_dataloader = setup_dataloaders(train_type=DatasetType.TRAIN)
model = CustomViT()
model, criterion, optimizer = setup_model(model, lr, device)
model_handler = ModelFilesHandler()
measurer = MeasureToolFactory.get_measure_tool(ModelType.TRANSFORMER)


train(criterion, device, epochs, model, optimizer, train_dataloader, model_handler, log_batches=True)
measures = evaluate(criterion, device, model, test_dataloader, measurer)

Epoch: 0


C:\Users\mpilc\Desktop\Studia\Thesis\Repozytoria\Thesis\venv\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


[=========================-------------------------] 50.00% [Loss: 0.760143]Epoch 1, Loss: 0.6893462008900113
Validation loss: 0.5071701407432556.
Measures:
         TP        FP        FN        TN  Precision  Sensitivity  \
0  0.249267  0.014663  0.237537  0.498534   0.944441     0.512047   

   Specificity      NPV       FPR  Accuracy  F-Score       IoU       MCC  \
0     0.971427  0.67729  0.028571    0.7478  0.66406  0.497076  0.548263   

        AUC        CI  
0  0.741738  0.046851  


# Save model

In [9]:
model_handler.save_model(
    model=model,
    metrics = measures,
    model_type=ModelType.TRANSFORMER_CLASSIFIER,
    epoch=epochs,
)

'trained_models\\model_transformer_classifier_2024_12_14_12_05_29.pickle'